In [10]:
urls = [
    (2013, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2013/Anexo_I_final_2013.xlsx"),
    (2014, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2014/Anexo_I_final_2014.xlsx"),
    (2015, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2015/2015_resultados_finais_censo_escolar_anexo_I.xlsx"),
    (2016, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2016/retificacao_anexo_I.xlsx"),
    (2017, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2017/dados_2017_Final_Anexo_I.xlsx"),
    (2018, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2018/2018_final_anexo_I.xlsx"),
    (2019, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2019/anexo_I_final-2019.xlsx"),
    (2020, "https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2020/Anexo_I_2020_Final.xlsx"),
    (2021, "https://download.inep.gov.br/censo_escolar/resultados/2021/resultado_final_anexo_1_.xlsx"),
    (2022, "https://download.inep.gov.br/censo_escolar/resultados/2022/DOU_final_anexo_I.xlsx"),
    (2023, "https://download.inep.gov.br/censo_escolar/resultados/2023/resultados_censo_escolar_final_anexo_I.xlsx"),
    (2024, "https://download.inep.gov.br/censo_escolar/resultados/2024/resultados_censo_escolar_final_anexo_I.xlsx")
]

for year, url in urls:
    !wget {url} -O 'files/{year}.xlsx'

--2025-12-03 17:18:36--  https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2013/Anexo_I_final_2013.xlsx
Resolvendo download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Conectando-se a download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 2515122 (2.4M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Salvando em: ‘files/2013.xlsx’

files/2013.xlsx     100%[===================>]   2.40M  1.84MB/s    em 1.3s    

2025-12-03 17:18:37 (1.84 MB/s) - ‘files/2013.xlsx’ salvo [2515122/2515122]

--2025-12-03 17:18:37--  https://download.inep.gov.br/educacao_basica/censo_escolar/resultado/2014/Anexo_I_final_2014.xlsx
Resolvendo download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Conectando-se a download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 2519563 (2.

In [29]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm.auto import tqdm

In [25]:
files = glob("files/*")
len(files)

12

In [60]:
import pandas as pd
import numpy as np

def tabela_municipios_ensino_regular(df: pd.DataFrame) -> pd.DataFrame:
    # 1. Descobrir onde está a linha "Matrícula inicial" (começo dos cabeçalhos das colunas numéricas)
    col_matricula_raw = df.columns[df.eq('Matrícula inicial').any()][0]
    idx_matricula = df.index[df[col_matricula_raw].eq('Matrícula inicial')][0]

    # Linhas que compõem o cabeçalho de múltiplos níveis (ajuste o +4 se necessário)
    header_rows = df.iloc[idx_matricula:idx_matricula+4]

    # Vamos guardar as partes do cabeçalho de cada coluna
    header_parts = {}
    new_cols = {}

    for col in df.columns:
        # Não mexer nas colunas com texto (onde ficam Brasil/UF/municípios)
        if col in ['Unnamed: 0', 'Unnamed: 1']:
            continue

        parts = [v for v in header_rows[col].tolist() if pd.notna(v)]
        if parts:
            parts_clean = [str(p).strip() for p in parts]
            header_parts[col] = parts_clean
            new_cols[col] = "_".join(parts_clean)

    # Renomear colunas numéricas
    df = df.rename(columns=new_cols)
    # Mapa reverso: nome_novo -> nome_original
    reverse_cols = {v: k for k, v in new_cols.items()}

    # Nome da coluna de matrícula após renomear
    col_matricula = new_cols.get(col_matricula_raw, col_matricula_raw)

    # 3. Cortar o cabeçalho "textual" de cima e começar a partir de BRASIL
    col_local = 'Unnamed: 1'  # nesta coluna aparecem BRASIL, UFs e municípios
    start_idx = df.index[df[col_local].eq('BRASIL')][0]
    data = df.iloc[start_idx:].copy()

    # 4. Criar uma coluna "place" (Brasil / UF / Município), preenchendo por forward-fill
    is_place = data[col_local].notna() & data[col_matricula].isna()
    data['place'] = np.where(is_place, data[col_local], np.nan)
    data['place'] = data['place'].ffill()

    # --- Estados por nome completo (com e sem acento) ---
    estados = {
        'ACRE', 'ALAGOAS', 'AMAPÁ', 'AMAPA', 'AMAZONAS', 'BAHIA', 'CEARÁ', 'CEARA',
        'DISTRITO FEDERAL', 'ESPÍRITO SANTO', 'ESPIRITO SANTO', 'GOIÁS', 'GOIAS',
        'MARANHÃO', 'MARANHAO', 'MATO GROSSO', 'MATO GROSSO DO SUL',
        'MINAS GERAIS', 'PARÁ', 'PARA', 'PARAÍBA', 'PARAIBA', 'PARANÁ', 'PARANA',
        'PERNAMBUCO', 'PIAUÍ', 'PIAUI', 'RIO DE JANEIRO', 'RIO GRANDE DO NORTE',
        'RIO GRANDE DO SUL', 'RONDÔNIA', 'RONDONIA', 'RORAIMA', 'SANTA CATARINA',
        'SÃO PAULO', 'SAO PAULO', 'SERGIPE', 'TOCANTINS'
    }

    uf_map = {
        'ACRE': 'AC',
        'ALAGOAS': 'AL',
        'AMAPÁ': 'AP', 'AMAPA': 'AP',
        'AMAZONAS': 'AM',
        'BAHIA': 'BA',
        'CEARÁ': 'CE', 'CEARA': 'CE',
        'DISTRITO FEDERAL': 'DF',
        'ESPÍRITO SANTO': 'ES', 'ESPIRITO SANTO': 'ES',
        'GOIÁS': 'GO', 'GOIAS': 'GO',
        'MARANHÃO': 'MA', 'MARANHAO': 'MA',
        'MATO GROSSO': 'MT',
        'MATO GROSSO DO SUL': 'MS',
        'MINAS GERAIS': 'MG',
        'PARÁ': 'PA', 'PARA': 'PA',
        'PARAÍBA': 'PB', 'PARAIBA': 'PB',
        'PARANÁ': 'PR', 'PARANA': 'PR',
        'PERNAMBUCO': 'PE',
        'PIAUÍ': 'PI', 'PIAUI': 'PI',
        'RIO DE JANEIRO': 'RJ',
        'RIO GRANDE DO NORTE': 'RN',
        'RIO GRANDE DO SUL': 'RS',
        'RONDÔNIA': 'RO', 'RONDONIA': 'RO',
        'RORAIMA': 'RR',
        'SANTA CATARINA': 'SC',
        'SÃO PAULO': 'SP', 'SAO PAULO': 'SP',
        'SERGIPE': 'SE',
        'TOCANTINS': 'TO',
    }

    # --- Atribuir UF (nome completo) a cada linha ---
    ufs = []
    current_uf = None
    for _, row in data.iterrows():
        label = row[col_local]

        if pd.isna(label):
            ufs.append(current_uf)
            continue

        if label == 'BRASIL':
            current_uf = None
        elif label in estados:
            current_uf = label  # nome completo

        ufs.append(current_uf)

    data['uf'] = ufs

    # 5. Manter apenas as linhas "Estadual e Municipal" de cada bloco
    mask_total_dep = data[col_local].eq('Estadual e Municipal')
    totais = data.loc[mask_total_dep].copy()

    # 6. Selecionar colunas de Ensino Regular usando o cabeçalho, NÃO só pelo nome da coluna
    regular_cols = []
    termos_regular = [
        'EDUCAÇÃO INFANTIL', 'EDUCACAO INFANTIL',
        'ENSINO FUNDAMENTAL',
        'FUNDAMENTAL',
        'MÉDIO', 'MEDIO',
        'CRECHE', 'PRÉ-ESCOLA', 'PRE-ESCOLA',
        'ANOS INICIAIS', 'ANOS FINAIS'
    ]

    for col in totais.columns:
        # ignorar colunas não renomeadas / não numéricas
        if col not in reverse_cols:
            continue

        orig = reverse_cols[col]
        parts = header_parts.get(orig, [])
        if not parts:
            continue

        header_txt = " ".join(parts).upper()

        # excluir EJA
        if 'EJA' in header_txt:
            continue

        # incluir apenas se bater com termos de ensino regular
        if any(t in header_txt for t in termos_regular):
            regular_cols.append(col)

    # 7. Calcular o total de matrículas em Ensino Regular (Infantil + Fundamental + Médio)
    totais['matriculas_ensino_regular_total'] = (
        totais[regular_cols]
        .apply(pd.to_numeric, errors='coerce')
        .sum(axis=1)
    )

    # 8. Remover Brasil e UFs — ficar só com municípios
    excluir = estados | {'BRASIL'}
    municipios = totais[~totais['place'].isin(excluir)].copy()

    # 9. Converter UF de nome completo para sigla
    municipios['uf'] = municipios['uf'].map(uf_map)

    # 10. Construir tabela final
    out = municipios[['place', 'uf', 'matriculas_ensino_regular_total']].rename(
        columns={'place': 'municipio'}
    )
    out = out.reset_index(drop=True)
    return out


In [61]:
data = []

for file in tqdm(files):
    df = pd.read_excel(file)
    result = tabela_municipios_ensino_regular(df)

    year = int(file.split("/")[-1].replace(".xlsx", ""))
    result["ano"] = year

    data.append(result)

data = pd.concat(data)

  0%|          | 0/12 [00:00<?, ?it/s]

In [62]:
data = (
    data
    .sort_values(['municipio', 'uf', 'ano'], ascending=[True, True, True])
    .reset_index(drop=True)
)

data['semestre'] = None

data

,municipio,uf,matriculas_ensino_regular_total,ano,semestre
0,ABADIA DE GOIAS,GO,2040,2013,None
1,ABADIA DE GOIAS,GO,2030,2014,None
2,ABADIA DE GOIAS,GO,2062,2015,None
3,ABADIA DE GOIAS,GO,2378,2016,None
4,ABADIA DE GOIAS,GO,2660,2017,None
...,...,...,...,...,...
66727,ZORTEA,SC,808,2020,None
66728,ZORTEA,SC,880,2021,None
66729,ZORTEA,SC,792,2022,None
66730,ZORTEA,SC,877,2023,None


In [70]:
print(data)

             municipio  uf  matriculas_ensino_regular_total   ano semestre
0      ABADIA DE GOIAS  GO                             2040  2013     None
1      ABADIA DE GOIAS  GO                             2030  2014     None
2      ABADIA DE GOIAS  GO                             2062  2015     None
3      ABADIA DE GOIAS  GO                             2378  2016     None
4      ABADIA DE GOIAS  GO                             2660  2017     None
...                ...  ..                              ...   ...      ...
66727           ZORTEA  SC                              808  2020     None
66728           ZORTEA  SC                              880  2021     None
66729           ZORTEA  SC                              792  2022     None
66730           ZORTEA  SC                              877  2023     None
66731           ZORTEA  SC                              748  2024     None

[66732 rows x 5 columns]


In [64]:
data.sort_values("matriculas_ensino_regular_total").head(20)

,municipio,uf,matriculas_ensino_regular_total,ano,semestre
28223,ITAOCA,SP,0,2024,None
31211,JOCA CLAUDINO,PB,0,2024,None
24143,GUARANI D OESTE,SP,0,2024,None
44483,PEDRINHAS PAULISTA,SP,0,2024,None
21981,FORTALEZA DO TABOCAO,TO,0,2022,None
21983,FORTALEZA DO TABOCAO,TO,0,2024,None
40064,NOVA LUZITANIA,SP,0,2021,None
52307,SANTA CLARA D OESTE,SP,0,2024,None
39683,NOVA CASTILHO,SP,0,2024,None
64607,URU,SP,0,2024,None


In [68]:
hab = pd.read_csv("SIDRA_Habitantes.csv")
print(hab)

                                Nivel     Codigo                   Nome  \
0                                  MU  1100015.0  Alta Floresta D'Oeste   
1                                  MU  1100023.0              Ariquemes   
2                                  MU  1100031.0                 Cabixi   
3                                  MU  1100049.0                 Cacoal   
4                                  MU  1100056.0             Cerejeiras   
...                               ...        ...                    ...   
5566                               MU  5222054.0         Vicentinópolis   
5567                               MU  5222203.0               Vila Boa   
5568                               MU  5222302.0          Vila Propício   
5569                               MU  5300108.0               Brasília   
5570  Fonte: IBGE - Censo Demográfico        NaN                    NaN   

        Num Hab  
0       21494.0  
1       96833.0  
2        5351.0  
3       86887.0  
4       1

In [72]:
import pandas as pd
import numpy as np
import unicodedata
import re

def normalizar_nome(s: str) -> str:
    if pd.isna(s):
        return None
    s = str(s).strip().upper()
    # remove acentos
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    # remove tudo que não for letra ou número (espaço, apóstrofo, hífen etc.)
    s = re.sub(r"[^A-Z0-9]", "", s)
    return s

def tabela_taxa_matriculas_100k(data: pd.DataFrame, hab: pd.DataFrame) -> pd.DataFrame:
    # --- 1. Limpar tabela de habitantes (IBGE) ---
    hab_clean = hab.copy()

    # manter apenas municípios (Nivel == "MU")
    hab_clean = hab_clean[hab_clean["Nivel"] == "MU"].copy()

    # remover linhas sem código ou população
    hab_clean = hab_clean[hab_clean["Num Hab"].notna() & hab_clean["Codigo"].notna()].copy()

    # criar coluna 'codigo' como inteiro
    hab_clean["codigo"] = hab_clean["Codigo"].astype(int)

    # chave normalizada de nome
    hab_clean["nome_norm"] = hab_clean["Nome"].apply(normalizar_nome)

    # --- 2. Preparar tabela de matrículas ---
    matr = data.copy()

    # chave normalizada de município
    matr["municipio_norm"] = matr["municipio"].apply(normalizar_nome)

    # --- 3. Merge: matrículas x população ---
    merged = matr.merge(
        hab_clean[["nome_norm", "codigo", "Num Hab"]],
        left_on="municipio_norm",
        right_on="nome_norm",
        how="left"
    )

    # --- 4. Calcular taxa por 100 mil habitantes ---
    # evitar divisão por zero
    denom = merged["Num Hab"].replace({0: np.nan})

    merged["taxa_matriculas_100k"] = (
        merged["matriculas_ensino_regular_total"] / denom * 100000
    )

    # opcional: arredondar
    merged["taxa_matriculas_100k"] = merged["taxa_matriculas_100k"].round(2)

    # --- 5. Montar tabela final ---
    final = merged[["municipio", "codigo", "uf", "ano", "semestre", "taxa_matriculas_100k"]].copy()

    # opcional: ordenar
    final = final.sort_values(["uf", "municipio", "ano"]).reset_index(drop=True)

    return final

# Uso:
tabela_final = tabela_taxa_matriculas_100k(data, hab)
print(tabela_final.head(20))


       municipio     codigo  uf   ano semestre  taxa_matriculas_100k
0     ACRELANDIA  1200013.0  AC  2013     None              30696.81
1     ACRELANDIA  1200013.0  AC  2014     None              30247.49
2     ACRELANDIA  1200013.0  AC  2015     None              25725.70
3     ACRELANDIA  1200013.0  AC  2016     None              27893.87
4     ACRELANDIA  1200013.0  AC  2017     None              29598.46
5     ACRELANDIA  1200013.0  AC  2018     None              27786.89
6     ACRELANDIA  1200013.0  AC  2019     None              25369.09
7     ACRELANDIA  1200013.0  AC  2020     None              27465.94
8     ACRELANDIA  1200013.0  AC  2021     None              28086.44
9     ACRELANDIA  1200013.0  AC  2022     None              26296.27
10    ACRELANDIA  1200013.0  AC  2023     None              23329.29
11    ACRELANDIA  1200013.0  AC  2024     None              20333.79
12  ASSIS BRASIL  1200054.0  AC  2013     None              37716.05
13  ASSIS BRASIL  1200054.0  AC  2

In [73]:
tabela_final.to_csv("education_indicator.csv")